In [225]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [226]:
import torch

In [227]:
import sys
sys.path.insert(0, '/media/mistertandon/DATA/git_repos/ai/01-llm-rsbt/ch_02')

from ch02_26_data_sampling import GPTDatasetV1, create_dataloader_v1

In [228]:
import sys
sys.path.insert(0, '/media/mistertandon/DATA/git_repos/ai/01-llm-rsbt/ch_03')

from ch_03_3_1_simple_self_attention import naive_softmax

In [229]:
vocab_size = 50257
out_dim = 3
token_embedding_layer = torch.nn.Embedding(num_embeddings=vocab_size, embedding_dim=out_dim)
print(token_embedding_layer)

Embedding(50257, 3)


In [230]:
raw_text = "millionaire journey start with first step"

In [231]:
dataloader = create_dataloader_v1(raw_text, batch_size=1, max_length=6, stride=6, shuffle=False, drop_last=False, num_workers=0)
data_iter = iter(dataloader)
print(data_iter)

In [232]:
inputs, targets = next(data_iter)

In [233]:
print(inputs)

tensor([[14100,  7626,  7002,   923,   351,   717]])


In [234]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([1, 6, 3])


In [235]:
token_embeddings[0, 0]

tensor([-0.4987,  0.5730, -0.9111], grad_fn=<SelectBackward0>)

In [236]:
position_embedding_layer = torch.nn.Embedding(6, 3)
print(position_embedding_layer)


Embedding(6, 3)


In [237]:
positional_embeddings = position_embedding_layer(torch.arange(6))
print(positional_embeddings.shape)

torch.Size([6, 3])


In [238]:
_inputs_embeddings = token_embeddings + positional_embeddings
inputs_embeddings = _inputs_embeddings.squeeze(0)
print(inputs_embeddings.shape)
print(inputs_embeddings)

torch.Size([6, 3])
tensor([[-0.5182,  1.4178, -1.8611],
        [ 0.5967,  2.1435, -0.6472],
        [ 2.9290, -1.7107, -0.0911],
        [-1.9147, -0.1546,  0.0716],
        [-0.2451, -0.3226,  1.8209],
        [ 1.2715,  2.2679, -1.9439]], grad_fn=<SqueezeBackward1>)


No we'll try to compute attention score corresponding to q^2 i.e. journey

attention_score_2 is un-normalized attention score represneted by omega for 2nd element (At index 1)

In [239]:
query = inputs_embeddings[1]
attention_score_2 = torch.empty(inputs_embeddings.shape[0])
for idx, inputs_embedding in enumerate(inputs_embeddings):
    attention_score_2[idx] = torch.dot(query, inputs_embedding)

print(attention_score_2)

tensor([ 3.9342,  5.3695, -1.8603, -1.5203, -2.0162,  6.8778],
       grad_fn=<CopySlices>)


attention_weight_2_normalized is normalized attention score (Known as attention weigth Represented by Alpha) for 2nd element (At index 1)

In [240]:
attention_weight_2_normalized = torch.softmax(attention_score_2, dim=0)
print(attention_weight_2_normalized)

tensor([4.1332e-02, 1.7362e-01, 1.2582e-04, 1.7678e-04, 1.0765e-04, 7.8464e-01],
       grad_fn=<SoftmaxBackward0>)


In [241]:
attention_weight_2_naive_normalized =naive_softmax(attention_score_2)
print(attention_weight_2_naive_normalized)
print(attention_weight_2_naive_normalized.sum())

tensor([4.1332e-02, 1.7362e-01, 1.2582e-04, 1.7678e-04, 1.0765e-04, 7.8464e-01],
       grad_fn=<DivBackward0>)
tensor(1., grad_fn=<SumBackward0>)


In [244]:
context_vec_2 = torch.zeros(query.shape)
for idx, input_embedding in enumerate(inputs_embeddings):
    context_vec_2 += attention_weight_2_naive_normalized[idx] * input_embedding
print(context_vec_2)

tensor([ 1.0798,  2.2099, -1.7143], grad_fn=<AddBackward0>)
